<h2>导入包</h2>

In [24]:
import os
import re
import logging
from datetime import datetime, date
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from lightgbm import Dataset
import matplotlib.pyplot as plt
from astral import LocationInfo
from astral.sun import sunrise, sunset, dawn, noon, dusk


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

<h2>读取数据</h2>

<h3>比赛数据</h3>

In [25]:
x_train = pd.read_csv("../data/A榜-训练集_分布式光伏发电预测_气象变量数据.csv", encoding="gbk")
y_train = pd.read_csv("../data/A榜-训练集_分布式光伏发电预测_实际功率数据.csv", encoding="gbk")
info_train = pd.read_csv("../data/A榜-训练集_分布式光伏发电预测_基本信息.csv", encoding="gbk")
x_test = pd.read_csv("../data/A榜-测试集_分布式光伏发电预测_气象变量数据.csv", encoding="gbk")
y_test = pd.read_csv("../data/submit_example.csv", encoding="utf-8")
info_test = pd.read_csv("../data/A榜-测试集_分布式光伏发电预测_基本信息.csv", encoding="gbk")

In [26]:
x_train = pd.merge(x_train, info_train[["光伏用户编号", "装机容量(kW)", "经度", "纬度"]], how="left", on="光伏用户编号")
x_train["时间"] = pd.to_datetime(x_train["时间"], utc=False).dt.tz_localize('Asia/Shanghai')
x_test = pd.merge(x_test, info_test[["光伏用户编号", "装机容量(kW)", "经度", "纬度"]], how="left", on="光伏用户编号")
x_test["时间"] = pd.to_datetime(x_test["时间"], utc=False).dt.tz_localize('Asia/Shanghai')

In [27]:
y_train = y_train.set_index(["光伏用户编号", "综合倍率", "时间"]).stack().reset_index().rename(columns={0:"target"})
y_train["level_3"] = y_train["level_3"].apply(lambda x: int(x[1:]))
y_train["时间"] = pd.to_datetime(y_train["时间"], utc=False).dt.tz_localize('Asia/Shanghai')
y_train["时间"] = y_train["时间"] + (y_train["level_3"] - 1) * 15 * pd.Timedelta(1, unit="minutes")
y_train = y_train.drop(columns=["level_3"])

y_test = y_test.set_index(["光伏用户编号", "综合倍率", "时间"]).stack().reset_index().rename(columns={0:"target"})
y_test["level_3"] = y_test["level_3"].apply(lambda x: int(x[1:]))
y_test["时间"] = pd.to_datetime(y_test["时间"], utc=False).dt.tz_localize('Asia/Shanghai')
y_test["时间"] = y_test["时间"] + (y_test["level_3"] - 1) * 15 * pd.Timedelta(1, unit="minutes")
y_test = y_test.drop(columns=["level_3"])

In [28]:
df_train = pd.merge(x_train, y_train, on=["光伏用户编号", "时间"], how="left")
df_test = pd.merge(x_test, y_test, on=["光伏用户编号", "时间"], how="left")
df = pd.concat([df_train, df_test], axis=0)
logging.info(df.columns)

2024-03-23 21:09:38,186 : INFO : Index(['光伏用户编号', '时间', '气压(Pa）', '相对湿度（%）', '云量', '10米风速（10m/s）', '10米风向（°)',
       '温度（K）', '辐照强度（J/m2）', '降水（m）', '100m风速（100m/s）', '100m风向（°)',
       '装机容量(kW)', '经度', '纬度', '综合倍率', 'target'],
      dtype='object')


<h3>外部数据</h3>

In [29]:
outer_files = os.listdir("../data/")
outer_files = [x for x in outer_files if re.match(r"^open-meteo-f\d\.csv$", x)]
outer_dfs = list()
for outer_file in outer_files:
    outer_df = pd.read_csv(os.path.join("../data", outer_file), skiprows=3)
    outer_df["time"] = pd.to_datetime(outer_df["time"], utc=False).dt.tz_localize('Asia/Shanghai')
    outer_df = outer_df.set_index("time", drop=True).asfreq("15T")
    outer_df = outer_df.interpolate(method="time", axis=0)

    outer_df = outer_df.reset_index(drop=False).rename(columns={"time": "时间"})
    outer_df["光伏用户编号"] = re.match(r"^open-meteo-(f\d)\.csv$", outer_file).group(1)    
    outer_dfs.append(outer_df)
outer_data = pd.concat(outer_dfs, axis=0)
logging.info(outer_data.columns)

2024-03-23 21:09:40,531 : INFO : Index(['时间', 'temperature_2m (°C)', 'relative_humidity_2m (%)',
       'dew_point_2m (°C)', 'apparent_temperature (°C)', 'precipitation (mm)',
       'rain (mm)', 'snowfall (cm)', 'snow_depth (m)',
       'weather_code (wmo code)', 'pressure_msl (hPa)',
       'surface_pressure (hPa)', 'cloud_cover (%)', 'cloud_cover_low (%)',
       'cloud_cover_mid (%)', 'cloud_cover_high (%)',
       'et0_fao_evapotranspiration (mm)', 'vapour_pressure_deficit (kPa)',
       'wind_speed_10m (km/h)', 'wind_speed_100m (km/h)',
       'wind_direction_10m (°)', 'wind_direction_100m (°)',
       'wind_gusts_10m (km/h)', 'soil_temperature_0_to_7cm (°C)',
       'soil_temperature_7_to_28cm (°C)', 'soil_temperature_28_to_100cm (°C)',
       'soil_temperature_100_to_255cm (°C)', 'soil_moisture_0_to_7cm (m³/m³)',
       'soil_moisture_7_to_28cm (m³/m³)', 'soil_moisture_28_to_100cm (m³/m³)',
       'soil_moisture_100_to_255cm (m³/m³)', '光伏用户编号'],
      dtype='object')


<h3>整合数据</h3>

In [30]:
df = pd.merge(df, outer_data[["光伏用户编号", "时间", "apparent_temperature (°C)", "surface_pressure (hPa)"]], how="left", on=["光伏用户编号", "时间"])
df["温度（K）"] = df["apparent_temperature (°C)"] + 273.15
df["气压(Pa）"] = df["surface_pressure (hPa)"] * 100
df = df.drop(columns=["apparent_temperature (°C)", "surface_pressure (hPa)"])

<h2>特征工程</h2>

<h3>时间特征</h3>

In [31]:
df["年"] = df["时间"].dt.year
df["季节"] = df["时间"].dt.quarter
df["月"] = df["时间"].dt.month
df["日"] = df["时间"].dt.day
df["周"] = df["时间"].dt.week
df["is_day"] = (df["时间"].dt.hour + 6) % 24 // 12
df["分"] = df["时间"].dt.minute // 15 + df["时间"].dt.hour * 4

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


In [32]:
# df["分_"] = df["分"].copy()
# df = pd.get_dummies(df, columns=["分_"], prefix_sep="")

<h3>根据日出时间日落时间计算时间段</h3>

In [33]:
def solar_time(current_time, dawn_time, sunrise_time, noon_time, sunset_time, dusk_time) -> int:
    """
    根据太阳判断当前时间段\n
    """
    if dawn_time < current_time < sunrise_time:
        return 1
    elif sunrise_time <= current_time < noon_time:
        return 2
    elif noon_time <= current_time < sunset_time:
        return 3
    elif sunset_time <= current_time < dusk_time:
        return 4
    else:
        return 0

In [34]:
# df["地点"] = df.apply(lambda x: LocationInfo(name=x["光伏用户编号"], region="China", timezone="Asia/Shanghai", latitude=x["纬度"], longitude=x["经度"]), axis=1)
# df["黎明时刻"] = df.apply(lambda x: dawn(x["地点"].observer, date=x["时间"], tzinfo=x["地点"].timezone), axis=1).dt.tz_convert("Asia/Shanghai")
# df["日出时刻"] = df.apply(lambda x: sunrise(x["地点"].observer, date=x["时间"], tzinfo=x["地点"].timezone), axis=1).dt.tz_convert("Asia/Shanghai")
# df["正午时刻"] = df.apply(lambda x: noon(x["地点"].observer, date=x["时间"], tzinfo=x["地点"].timezone), axis=1).dt.tz_convert("Asia/Shanghai")
# df["日落时刻"] = df.apply(lambda x: sunset(x["地点"].observer, date=x["时间"], tzinfo=x["地点"].timezone), axis=1).dt.tz_convert("Asia/Shanghai")
# df["黄昏时刻"] = df.apply(lambda x: dusk(x["地点"].observer, date=x["时间"], tzinfo=x["地点"].timezone), axis=1).dt.tz_convert("Asia/Shanghai")
# df["时间段"] = df.apply(lambda x: solar_time(x["时间"], x["黎明时刻"], x["日出时刻"], x["正午时刻"], x["日落时刻"], x["黄昏时刻"]), axis=1)
# df = df.drop(columns=["地点", "黎明时刻", "日出时刻", "正午时刻", "日落时刻", "黄昏时刻"])

<h3>光伏用户编号</h3>

In [35]:
df["光伏用户编号_"] = df["光伏用户编号"].copy()
df = pd.get_dummies(df, columns=["光伏用户编号_"], prefix_sep="")

<h3>气象特征</h3>

In [36]:
df['100m风速（100m/s）'] = df['100m风速（100m/s）'] * np.sin(np.pi * df['100m风向（°)'] / 180)
# df['cos_100m风速（100m/s）'] = df['100m风速（100m/s）'] * np.cos(np.pi * df['100m风向（°)'] / 180)
df['10米风速（10m/s）'] = df['10米风速（10m/s）'] * np.sin(np.pi * df['10米风向（°)'] / 180)
# df['cos_10米风速（10m/s）'] = df['10米风速（10m/s）'] * np.cos(np.pi * df['10米风向（°)'] / 180)

<h3>历史值特征</h3>

In [37]:
dfs = []
for site, df_site in df.groupby("光伏用户编号"):
    df_site = df_site.sort_values("时间")
    df_site["辐照强度（J/m2） - 1"] = df_site["辐照强度（J/m2）"].shift(1) - df_site["辐照强度（J/m2）"]
    df_site["辐照强度（J/m2） - 8"] = df_site["辐照强度（J/m2）"].shift(8) - df_site["辐照强度（J/m2）"]
#     df_site["辐照强度（J/m2） - 2"] = df_site["辐照强度（J/m2）"].shift(2) - df_site["辐照强度（J/m2）"]
    dfs.append(df_site)
df = pd.concat(dfs, axis=0)

<h3>处理异常值</h3>

In [38]:
print(df_train["target"].nsmallest(3))
df_train[df_train["target"] < -8]
df.loc[207628:207633, ["光伏用户编号", "时间", "target"]]

207630   -8.8900
37316    -0.0085
39139    -0.0084
Name: target, dtype: float64


,光伏用户编号,时间,target
207628,f6,2022-08-15 20:30:00+08:00,-0.002
207629,f6,2022-08-15 20:45:00+08:00,-0.002
207630,f6,2022-08-15 21:00:00+08:00,-8.890
207631,f6,2022-08-15 21:15:00+08:00,-0.002
207632,f6,2022-08-15 21:30:00+08:00,NaN
207633,f6,2022-08-15 21:45:00+08:00,NaN


In [39]:
df.loc[207630, "target"] = -0.002

<h3>光照与当天最强光照的比值</h3>

In [40]:
df["日期"] = df["时间"].dt.date
day_max_values = df[["光伏用户编号", "日期", "辐照强度（J/m2）"]].groupby(by=["光伏用户编号", "日期"]).max()
day_max_values = day_max_values.rename(columns={x: x + "_max" for x in day_max_values.columns}).reset_index()
df = pd.merge(df, day_max_values, on=["光伏用户编号", "日期"], how="left").drop(columns=["日期"])
df["辐照强度（J/m2）_max"] = df["辐照强度（J/m2）"] / df["辐照强度（J/m2）_max"]

<h3>当天的平均光照</h3>

In [41]:
df["日期"] = df["时间"].dt.date
day_mean_values = df[["光伏用户编号", "日期", "辐照强度（J/m2）"]].groupby(by=["光伏用户编号", "日期"]).mean()
day_mean_values = day_mean_values.rename(columns={x: x + "_mean" for x in day_mean_values.columns}).reset_index()
df = pd.merge(df, day_mean_values, on=["光伏用户编号", "日期"], how="left").drop(columns=["日期"])

<h3>温度与当天最高温最低度的差值</h3>

In [42]:
df["日期"] = df["时间"].dt.date
day_max_values = df[["光伏用户编号", "日期", "温度（K）"]].groupby(by=["光伏用户编号", "日期"]).max()
day_min_values = df[["光伏用户编号", "日期", "温度（K）"]].groupby(by=["光伏用户编号", "日期"]).min()
day_max_values = day_max_values.rename(columns={x: x + "_max" for x in day_max_values.columns}).reset_index()
day_min_values = day_min_values.rename(columns={x: x + "_min" for x in day_min_values.columns}).reset_index()
df = pd.merge(df, day_max_values, on=["光伏用户编号", "日期"], how="left")
df = pd.merge(df, day_min_values, on=["光伏用户编号", "日期"], how="left").drop(columns=["日期"])
df["温度（K）_max"] = df["温度（K）_max"] - df["温度（K）"]
df["温度（K）_min"] = df["温度（K）"] - df["温度（K）_min"]

<h3>划分测试集</h3>

In [43]:
df_train = df[df["时间"] <= df_train["时间"].max()]
df_test = df[df["时间"] >= df_test["时间"].min()]

<h2>训练模型</h2>

<h3>评测指标</h3>

In [44]:
def score(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return 1 / (1 + rmse)

<h3>lightgbm模型</h3>

In [45]:
params_lgb = {
    "num_boost_round": 1000,
    'learning_rate': 0.02,
    'boosting_type': 'gbdt',
    'objective': 'mse',
    'metric': 'mse',
    'num_leaves': 127,
    'verbose': -1,
    'seed': 42,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    "early_stopping_round": 100
}

<h3>交叉验证</h3>

In [46]:
model_lgb = []
kfold = KFold(n_splits=5, random_state=42, shuffle=True)

x = df_train.drop(columns=["光伏用户编号", "时间"]).astype(np.float32)
y = x.pop("target")
mse = 0
for fold, (train_index, val_index) in enumerate(kfold.split(x, y)):
    logging.info(f'############ fold: {fold} ###########')
    x_train, x_val, y_train, y_val = x.iloc[train_index], x.iloc[val_index], y.iloc[train_index], y.iloc[val_index]
    trainset = Dataset(x_train, y_train)
    valset = Dataset(x_val, y_val)
    model = lgb.train(params_lgb, trainset, valid_sets=[trainset, valset], categorical_feature=["分"], callbacks=[lgb.log_evaluation(1000)])
    model.save_model("../models/lgb_%d.txt" % fold)
    model_lgb.append(model)
    val_pred = Series(model.predict(x_val, num_iteration=model.best_iteration), index=y_val.index).fillna(0)
    mse += mean_squared_error(y_val.fillna(0), val_pred)
rmse = np.sqrt(mse)
score = 1 / (1 + rmse)
logging.info(f"--------------本地分数 {score}--------------")

2024-03-23 21:09:43,328 : INFO : ############ fold: 0 ###########
c:\program files\python37\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\program files\python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['分']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\program files\python37\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
c:\program files\python37\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1000]	training's l2: 0.0133533	valid_1's l2: 0.0217172


2024-03-23 21:10:02,048 : INFO : ############ fold: 1 ###########
c:\program files\python37\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\program files\python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['分']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\program files\python37\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
c:\program files\python37\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1000]	training's l2: 0.0135043	valid_1's l2: 0.0217567


2024-03-23 21:10:20,872 : INFO : ############ fold: 2 ###########
c:\program files\python37\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\program files\python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['分']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\program files\python37\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
c:\program files\python37\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


...

2024-03-23 21:10:39,551 : INFO : ############ fold: 3 ###########
c:\program files\python37\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\program files\python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['分']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\program files\python37\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
c:\program files\python37\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1000]	training's l2: 0.0131479	valid_1's l2: 0.0214833


2024-03-23 21:10:58,192 : INFO : ############ fold: 4 ###########
c:\program files\python37\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\program files\python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['分']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\program files\python37\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
c:\program files\python37\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1000]	training's l2: 0.0135691	valid_1's l2: 0.0216446


2024-03-23 21:11:16,828 : INFO : --------------本地分数 0.7529043147099884--------------


In [47]:
importance = DataFrame()
importance["特征"] = model.feature_name()
importance["重要性"] = 0
for model in model_lgb:
    importance["重要性"] = importance["重要性"] + model.feature_importance()
importance["重要性"] = importance["重要性"] / kfold.n_splits
importance.sort_values("重要性", ascending=False)[0:50]

,特征,重要性
33,辐照强度（J/m2）_mean,10310.0
0,气压(Pa）,9598.8
1,相对湿度（%）,9231.8
7,降水（m）,9200.4
5,温度（K）,7869.2
35,温度（K）_min,7738.4
17,日,7479.0
6,辐照强度（J/m2）,5592.6
18,周,5446.4
31,辐照强度（J/m2）_-_8,5331.2


<h2>预测</h2>

In [48]:
x_test = df_test.drop(columns=["光伏用户编号", "时间"]).astype(np.float32)
y_test = x_test.pop("target")
y_pred = np.zeros((df_test.shape[0], ))
for model in model_lgb:
    y_pred += model.predict(x_test, num_iteration=model.best_iteration)
y_pred = y_pred / kfold.n_splits
df_test["target"] = y_pred

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [49]:
df_test = df_test[["光伏用户编号", "综合倍率", "年", "月", "日", "分", "target"]]
df_test["时间"] = df_test["年"].astype(str) + "-" + df_test["月"].astype(str) + "-" + df_test["日"].astype(str) + " 0:00"
df_test["分"] = "p" + (df_test["分"] + 1).astype(str)
df_test = df_test.drop(columns=["年", "月", "日"])

In [50]:
result = pd.pivot(df_test, index=["光伏用户编号", "综合倍率", "时间"], columns="分", values="target").reset_index()
result = result[result["综合倍率"].notnull()]
result["综合倍率"] = result["综合倍率"].astype(int)

In [51]:
result.to_csv("../data/%s.csv" % datetime.now().strftime("%Y%m%d_%H%M%S"), encoding="utf-8", index=False)